In [10]:
# import des bibliothèques nécessaires
import cv2

# Chemin vers le fichier vidéo  
video_path = r'C:\Users\habib\Desktop\Montages volley et beach\montage_auto\video.mp4'

# Vitesse de lecture de la vidéo (1=normale, 0=pause, >1=plus rapide)
play_speed = 2.5


Encodage en direct des 'Points' et 'Temps mort' à partir de la lecture de la vidéo

In [ ]:
# Liste pour stocker les actions
actions = []

# Dictionnaire des touches -> action
key_action_map = {
    ord('1'): 'Point',
    ord('2'): 'Temps hors-jeu',
}

# Ouvrir la vidéo
cap = cv2.VideoCapture(video_path)

def _waitKey_fast(ms):
    # réduire le délai proportionnellement à la vitesse (au moins 1 ms)
    adj = max(1, int(ms / play_speed))
    return cv2.waitKey(adj)

if not cap.isOpened():
    print("Erreur : impossible d’ouvrir la vidéo.")
    exit()

fps = cap.get(cv2.CAP_PROP_FPS)
frame_number = 0

print("\n--- LECTURE EN COURS ---\nAppuie sur une touche pour taguer une action : 1/2 (ou q pour quitter)\n")
paused = False
last_action = ""  # Variable pour stocker la dernière action

try:
    while cap.isOpened():
        if not paused:
            ret, frame = cap.read()
            if not ret:
                print("Fin de la vidéo ou erreur de lecture.")
                break
            frame_number += 1

        # Affiche la dernière action
        if last_action and ret:
            cv2.putText(frame,
                        f"Derniere action : {last_action}",
                        (30, 40),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,
                        (0, 255, 0),
                        2,
                        cv2.LINE_AA)

        # Indiquer mode pause
        if paused and ret:
            cv2.putText(frame,
                        "|| PAUSE ||",
                        (30, 80),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,
                        (0, 0, 255),
                        2,
                        cv2.LINE_AA)

        if ret:
            cv2.imshow('Beach Volley Video', frame)


        key = _waitKey_fast(30) & 0xFF
        if key == ord('q'):  # touche 'q' pour quitter
            break
        elif key == ord(' '):  # barre espace
            paused = not paused
        elif key in key_action_map:
            timestamp = frame_number / fps
            timestamp_hhmmss = f"{int(timestamp // 3600):02d}:{int((timestamp % 3600) // 60):02d}:{int(timestamp % 60):02d}"
            action_name = key_action_map[key]
            last_action = action_name

            actions.append({
                # 'Time (MM:SS)': timestamp_hhmmss,
                'Frame': frame_number,
                # 'Time (s)': round(timestamp, 2),
                'Action': action_name
            })

finally:
    # Libérer les ressources
    cap.release()
    cv2.destroyAllWindows()


### Ecriture des intervalles en numéro de frames dans deux fichiers textes : 1 pour les points, 1 pour les temps hors-jeu

## Fichier 'Points.txt' (format : frame_start-frame_end)
points_file = 'Points_frames.txt'
# Construire les intervalles : 1 ligne = time(Point) - time(Temps hors-jeu) suivant
points_frames = []
i = 0
while i < len(actions):
    if actions[i]['Action'] == 'Point':
        start = actions[i]['Frame']
        # chercher le prochain 'Temps hors-jeu'
        j = i + 1
        end = None
        while j < len(actions):
            if actions[j]['Action'] == 'Temps hors-jeu':
                end = actions[j]['Frame']
                break
            j += 1
        if end is not None:
            points_frames.append((start, end))
            i = j + 1
        else:
            print(f"⚠️ Pas d'action 'Point' trouvée après {start:.2f}s — intervalle ignoré.")
            break
    else:
        i += 1
# Écrire les intervalles dans new_ranges.txt (format : start-end))
ranges_frames_file = 'new_ranges_frames.txt'
with open(points_file, 'w', encoding='utf-8') as f:

    for start, end in points_frames:
        f.write(f"{start}-{end}\n")
print(f"✅ Fichier texte sauvegardé : {points_file}")

## Fichier 'Temps_hors_jeu.txt' (format : frame_start-frame_end)
temps_hors_jeu_file = 'Temps_hors_jeu_frames.txt'
# Construire les intervalles : 1 ligne = time(Temps hors-jeu) - time(Point) suivant
temps_hors_jeu_frames = []
i = 0
while i < len(actions):
    if actions[i]['Action'] == 'Temps hors-jeu':
        start = actions[i]['Frame']
        # chercher le prochain 'Point'
        j = i + 1
        end = None
        while j < len(actions):
            if actions[j]['Action'] == 'Point':
                end = actions[j]['Frame']
                break
            j += 1
        if end is not None:
            temps_hors_jeu_frames.append((start, end))
            i = j + 1
        else:
            print(f"⚠️ Pas d'action 'Point' trouvée après {start:.2f}s — intervalle ignoré.")
            break
    else:
        i += 1
# Écrire les intervalles dans temps_hors_jeu_frames.txt (format : start-end)
with open(temps_hors_jeu_file, 'w', encoding='utf-8') as f:
    for start, end in temps_hors_jeu_frames:
        f.write(f"{start}-{end}\n")
print(f"✅ Fichier texte sauvegardé : {temps_hors_jeu_file}")





--- LECTURE EN COURS ---
Appuie sur une touche pour taguer une action : 1/2 (ou q pour quitter)

✅ Fichier texte sauvegardé : new_ranges_frames.txt


Découpage des segments à partir d'un fichier .txt contenant les paires de frames "Point-Temps mort"

In [3]:
import montage_auto_frames_gpu as montage_gpu

# Paramètres
video_path = r'C:\Users\habib\Desktop\Montages volley et beach\montage_auto\video.mp4'
ranges_frames_file = 'ranges.txt'
output_dir = r'C:\Users\habib\Desktop\Montages volley et beach\montage_auto\output_segments_gpu'

montage_gpu.extract_segments_gpu(video_path, ranges_frames_file, output_dir=output_dir)

[INFO] Nombre de segments à extraire : 46
[GPU] Extraction frames 1427 → 1526
      Command: ffmpeg -y -hwaccel cuda -hwaccel_output_format cuda -i C:\Users\habib\Desktop\Montages volley et beach\montage_auto\video.mp4 -vf select='between(n,1427,1526)',setpts=PTS-STARTPTS -an -c:v h264_nvenc -preset p4 -rc constqp -qp 18 C:\Users\habib\Desktop\Montages volley et beach\montage_auto\output_segments_gpu\extrait_001.mp4
[GPU] Extraction frames 1753 → 2033
      Command: ffmpeg -y -hwaccel cuda -hwaccel_output_format cuda -i C:\Users\habib\Desktop\Montages volley et beach\montage_auto\video.mp4 -vf select='between(n,1753,2033)',setpts=PTS-STARTPTS -an -c:v h264_nvenc -preset p4 -rc constqp -qp 18 C:\Users\habib\Desktop\Montages volley et beach\montage_auto\output_segments_gpu\extrait_002.mp4
[GPU] Extraction frames 2449 → 2741
      Command: ffmpeg -y -hwaccel cuda -hwaccel_output_format cuda -i C:\Users\habib\Desktop\Montages volley et beach\montage_auto\video.mp4 -vf select='between(n,244

True